In [1]:
!pip install pandas numpy sqlalchemy matplotlib openpyxl psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os

BASE = r"D:\Vaccination Project\Vaccination project"
folders = ["data", "outputs", "scripts"]

for f in folders:
    path = os.path.join(BASE, f)
    os.makedirs(path, exist_ok=True)
    print("Created:", path)

Created: D:\Vaccination Project\Vaccination project\data
Created: D:\Vaccination Project\Vaccination project\outputs
Created: D:\Vaccination Project\Vaccination project\scripts


In [8]:
for root, dirs, files in os.walk(BASE):
    level = root.replace(BASE, "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

Vaccination project/
    data/
        coverage-data.xlsx
        incidence-rate-data.xlsx
        reported-cases-data.xlsx
        vaccine-introduction-data.xlsx
        vaccine-schedule-data.xlsx
    outputs/
        coverage_clean.csv
        incidence_clean.csv
        reported_clean.csv
        vaccine_intro_clean.csv
        vaccine_schedule_clean.csv
    scripts/


In [9]:
import pandas as pd
import numpy as np
import re

DATA = os.path.join(BASE, "data")
OUT = os.path.join(BASE, "outputs")

FILES = {
    "coverage": os.path.join(DATA, "coverage-data.xlsx"),
    "incidence": os.path.join(DATA, "incidence-rate-data.xlsx"),
    "reported": os.path.join(DATA, "reported-cases-data.xlsx"),
    "intro": os.path.join(DATA, "vaccine-introduction-data.xlsx"),
    "schedule": os.path.join(DATA, "vaccine-schedule-data.xlsx"),
}

In [10]:
def snake(s: str) -> str:
    s = re.sub(r"\s+", "_", s.strip())
    s = re.sub(r"[^0-9a-zA-Z_]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s.lower()

def read_first_sheet(path: str) -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=0, engine="openpyxl")
    df.columns = [snake(str(c)) for c in df.columns]
    return df

def as_int(series):
    return pd.to_numeric(series, errors="coerce").astype("Int64")

def as_num(series):
    return pd.to_numeric(series, errors="coerce")

def map_columns(df, mapping):
    df = df.copy()
    for target, opts in mapping.items():
        for opt in opts:
            if opt in df.columns and target not in df.columns:
                df = df.rename(columns={opt: target})
                break
    return df

def standardize_country_cols(df):
    df = map_columns(df, {
        "iso3": ["iso_3_code","iso3","code"],
        "country_name": ["country_name","name","country"],
        "who_region": ["who_region","region"],
        "year": ["year"]
    })
    if "iso3" in df: df["iso3"] = df["iso3"].astype(str).str.upper().str.strip()
    if "year" in df: df["year"] = as_int(df["year"])
    return df

In [7]:
raw = {k: read_first_sheet(p) for k,p in FILES.items()}

cov = standardize_country_cols(raw["coverage"])
cov = map_columns(cov, {
    "antigen": ["antigen","vaccine"],
    "antigen_description": ["antigen_description","vaccine_description"],
    "coverage_category": ["coverage_category","category"],
    "coverage_category_description": ["coverage_category_description","category_description"],
    "target_number": ["target_number","target_pop"],
    "doses_administered": ["doses_administered","doses","dodge"],
    "coverage_pct": ["coverage","coverage_percent"],
})
for c in ["target_number","doses_administered","coverage_pct"]:
    if c in cov: cov[c] = as_num(cov[c])

inc = standardize_country_cols(raw["incidence"])
inc = map_columns(inc, {
    "disease": ["disease"],
    "disease_description": ["disease_description"],
    "denominator": ["denominator"],
    "incidence_rate": ["incidence_rate","rate"],
})
if "incidence_rate" in inc: inc["incidence_rate"] = as_num(inc["incidence_rate"])

rep = standardize_country_cols(raw["reported"])
rep = map_columns(rep, {
    "disease": ["disease"],
    "disease_description": ["disease_description"],
    "cases": ["cases","reported_cases"],
})
if "cases" in rep: rep["cases"] = as_num(rep["cases"])

intro = standardize_country_cols(raw["intro"])
intro = map_columns(intro, {
    "vaccine_description": ["description","vaccine_description"],
    "introduced": ["intro","introduced"],
})
if "introduced" in intro:
    intro["introduced"] = intro["introduced"].apply(
        lambda x: str(x).strip().lower() in {"1","true","yes","y"} if pd.notna(x) else np.nan
    )

sched = standardize_country_cols(raw["schedule"])
sched = map_columns(sched, {
    "vaccine_code": ["vaccine_code"],
    "vaccine_description": ["vaccine_description"],
    "schedule_rounds": ["schedule_rounds","dose","round"],
    "target_pop": ["target_pop"],
    "geoarea": ["geoarea"],
    "age_administered": ["age_administered"],
})

tables = {
    "coverage_clean": cov,
    "incidence_clean": inc,
    "reported_clean": rep,
    "vaccine_intro_clean": intro,
    "vaccine_schedule_clean": sched,
}

for name, df in tables.items():
    out = os.path.join(OUT, f"{name}.csv")
    df.to_csv(out, index=False)
    print("Saved:", out)

Saved: D:\Vaccination Project\Vaccination project\outputs\coverage_clean.csv
Saved: D:\Vaccination Project\Vaccination project\outputs\incidence_clean.csv
Saved: D:\Vaccination Project\Vaccination project\outputs\reported_clean.csv
Saved: D:\Vaccination Project\Vaccination project\outputs\vaccine_intro_clean.csv
Saved: D:\Vaccination Project\Vaccination project\outputs\vaccine_schedule_clean.csv


In [12]:
import sqlite3

DB = os.path.join(OUT, "vaccination.db")
tables = ["coverage_clean","incidence_clean","reported_clean",
          "vaccine_intro_clean","vaccine_schedule_clean"]

conn = sqlite3.connect(DB)

for t in tables:
    csv_path = os.path.join(OUT, f"{t}.csv")
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path, low_memory=False) 
        df.to_sql(t, conn, if_exists="replace", index=False)
        print("Loaded:", t)
    else:
        print("Missing file:", csv_path)

conn.commit()
conn.close()

print("SQLite DB created at:", DB)

Loaded: coverage_clean
Loaded: incidence_clean
Loaded: reported_clean
Loaded: vaccine_intro_clean
Loaded: vaccine_schedule_clean
SQLite DB created at: D:\Vaccination Project\Vaccination project\outputs\vaccination.db


In [13]:
conn = sqlite3.connect(DB)
q = "SELECT * FROM coverage_clean LIMIT 5;"
print(pd.read_sql(q, conn))
conn.close()

       group iso3 country_name    year  antigen  \
0  COUNTRIES  ABW        Aruba  2023.0      BCG   
1  COUNTRIES  ABW        Aruba  2023.0      BCG   
2  COUNTRIES  ABW        Aruba  2023.0  DIPHCV4   
3  COUNTRIES  ABW        Aruba  2023.0  DIPHCV4   
4  COUNTRIES  ABW        Aruba  2023.0  DIPHCV5   

                                 antigen_description coverage_category  \
0                                                BCG             ADMIN   
1                                                BCG          OFFICIAL   
2  Diphtheria-containing vaccine, 4th dose (1st b...             ADMIN   
3  Diphtheria-containing vaccine, 4th dose (1st b...          OFFICIAL   
4  Diphtheria-containing vaccine, 5th dose (2nd b...             ADMIN   

  coverage_category_description  target_number  doses_administered  \
0       Administrative coverage            NaN                 NaN   
1             Official coverage            NaN                 NaN   
2       Administrative coverage       

In [14]:
conn = sqlite3.connect(DB)
cur = conn.cursor()

tables = ["schedule","incidence","disease","vaccination","antigen","countries"]
for t in tables:
    cur.execute(f"DROP TABLE IF EXISTS {t};")

cur.execute("""
CREATE TABLE countries (
    iso3 TEXT PRIMARY KEY,
    country_name TEXT,
    who_region TEXT
);
""")

cur.execute("""
CREATE TABLE antigen (
    antigen TEXT PRIMARY KEY,
    antigen_description TEXT,
    coverage_category TEXT
);
""")

cur.execute("""
CREATE TABLE vaccination (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    iso3 TEXT,
    year INTEGER,
    antigen TEXT,
    coverage_pct REAL,
    doses_administered REAL,
    FOREIGN KEY (iso3) REFERENCES countries(iso3),
    FOREIGN KEY (antigen) REFERENCES antigen(antigen)
);
""")

cur.execute("""
CREATE TABLE disease (
    disease TEXT PRIMARY KEY,
    disease_description TEXT
);
""")

cur.execute("""
CREATE TABLE incidence (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    iso3 TEXT,
    year INTEGER,
    disease TEXT,
    incidence_rate REAL,
    cases REAL,
    FOREIGN KEY (iso3) REFERENCES countries(iso3),
    FOREIGN KEY (disease) REFERENCES disease(disease)
);
""")

cur.execute("""
CREATE TABLE schedule (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    iso3 TEXT,
    vaccine_description TEXT,
    schedule_rounds TEXT,
    age_administered TEXT,
    FOREIGN KEY (iso3) REFERENCES countries(iso3)
);
""")

conn.commit()
conn.close()
print(" Normalized tables created.")

 Normalized tables created.


In [18]:
conn = sqlite3.connect(DB)

conn.execute("""
INSERT OR IGNORE INTO countries (iso3, country_name)
SELECT DISTINCT iso3, country_name 
FROM coverage_clean;
""")

conn.execute("""
INSERT OR IGNORE INTO antigen (antigen, antigen_description, coverage_category)
SELECT DISTINCT antigen, antigen_description, coverage_category
FROM coverage_clean;
""")

conn.execute("""
INSERT INTO vaccination (iso3, year, antigen, coverage_pct, doses_administered)
SELECT iso3, year, antigen, coverage_pct, doses_administered
FROM coverage_clean;
""")

conn.commit()
conn.close()
print("Countries, Antigen, and Vaccination tables have been populated successfully.")

Countries, Antigen, and Vaccination tables have been populated successfully.


In [19]:
conn = sqlite3.connect(DB)
print(pd.read_sql("PRAGMA table_info(incidence_clean);", conn))
conn.close()

   cid                 name  type  notnull dflt_value  pk
0    0                group  TEXT        0       None   0
1    1                 iso3  TEXT        0       None   0
2    2         country_name  TEXT        0       None   0
3    3                 year  REAL        0       None   0
4    4              disease  TEXT        0       None   0
5    5  disease_description  TEXT        0       None   0
6    6          denominator  TEXT        0       None   0
7    7       incidence_rate  REAL        0       None   0


In [20]:
conn = sqlite3.connect(DB)
print(pd.read_sql("PRAGMA table_info(reported_clean);", conn))
conn.close()

   cid                 name  type  notnull dflt_value  pk
0    0                group  TEXT        0       None   0
1    1                 iso3  TEXT        0       None   0
2    2         country_name  TEXT        0       None   0
3    3                 year  REAL        0       None   0
4    4              disease  TEXT        0       None   0
5    5  disease_description  TEXT        0       None   0
6    6                cases  REAL        0       None   0


In [21]:
conn = sqlite3.connect(DB)

conn.execute("""
INSERT OR IGNORE INTO disease (disease, disease_description)
SELECT DISTINCT disease, disease_description
FROM incidence_clean;
""")

conn.execute("""
INSERT INTO incidence (iso3, year, disease, incidence_rate, cases)
SELECT iso3, year, disease, incidence_rate, denominator
FROM incidence_clean;
""")

conn.commit()
conn.close()
print(" Incidence_clean data inserted (diseases + incidence).")

 Incidence_clean data inserted (diseases + incidence).


In [22]:
conn = sqlite3.connect(DB)

conn.execute("""
INSERT OR IGNORE INTO disease (disease, disease_description)
SELECT DISTINCT disease, disease_description
FROM reported_clean;
""")

conn.execute("""
INSERT INTO incidence (iso3, year, disease, cases)
SELECT iso3, year, disease, cases
FROM reported_clean;
""")

conn.commit()
conn.close()
print(" Reported_clean data inserted (diseases + reported cases).")

 Reported_clean data inserted (diseases + reported cases).


In [5]:
import sqlite3, pandas as pd

conn = sqlite3.connect(DB)
print(pd.read_sql("PRAGMA table_info(vaccine_schedule_clean);", conn))
conn.close()

    cid                   name  type  notnull dflt_value  pk
0     0                   iso3  TEXT        0       None   0
1     1            countryname  TEXT        0       None   0
2     2             who_region  TEXT        0       None   0
3     3                   year  REAL        0       None   0
4     4            vaccinecode  TEXT        0       None   0
5     5    vaccine_description  TEXT        0       None   0
6     6         schedulerounds  REAL        0       None   0
7     7              targetpop  TEXT        0       None   0
8     8  targetpop_description  TEXT        0       None   0
9     9                geoarea  TEXT        0       None   0
10   10        ageadministered  TEXT        0       None   0
11   11          sourcecomment  TEXT        0       None   0


In [8]:
import sqlite3, pandas as pd

conn = sqlite3.connect(DB)

print(pd.read_sql("PRAGMA table_info(vaccine_schedule_clean);", conn))

print(pd.read_sql("SELECT * FROM vaccine_schedule_clean LIMIT 5;", conn))

conn.close()

    cid                   name  type  notnull dflt_value  pk
0     0                   iso3  TEXT        0       None   0
1     1            countryname  TEXT        0       None   0
2     2             who_region  TEXT        0       None   0
3     3                   year  REAL        0       None   0
4     4            vaccinecode  TEXT        0       None   0
5     5    vaccine_description  TEXT        0       None   0
6     6         schedulerounds  REAL        0       None   0
7     7              targetpop  TEXT        0       None   0
8     8  targetpop_description  TEXT        0       None   0
9     9                geoarea  TEXT        0       None   0
10   10        ageadministered  TEXT        0       None   0
11   11          sourcecomment  TEXT        0       None   0
  iso3 countryname who_region    year vaccinecode  \
0  ABW       Aruba       AMRO  2023.0  DTAPHIBIPV   
1  ABW       Aruba       AMRO  2023.0  DTAPHIBIPV   
2  ABW       Aruba       AMRO  2023.0  DTAPHIBIP

In [9]:
import sqlite3

conn = sqlite3.connect(DB)

conn.execute("""
INSERT INTO schedule (iso3, vaccine_description, schedule_rounds, age_administered)
SELECT iso3, vaccine_description, schedulerounds, ageadministered
FROM vaccine_schedule_clean;
""")

conn.commit()
conn.close()
print(" Schedule table populated successfully.")

 Schedule table populated successfully.


In [14]:
conn = sqlite3.connect(DB)
q = """
SELECT country_name, year, antigen, AVG(coverage_pct) AS avg_coverage
FROM vaccination v
JOIN countries c ON v.iso3 = c.iso3
GROUP BY country_name, year, antigen
ORDER BY year, avg_coverage DESC
LIMIT 20;
"""
print(pd.read_sql(q, conn))
conn.close()

            country_name    year antigen  avg_coverage
0                   None     NaN    None           NaN
1   China, Hong Kong SAR  1980.0     BCG    100.000000
2                 Panama  1980.0     BCG     99.666667
3               Barbados  1980.0    POL3     99.000000
4                Belgium  1980.0    POL3     99.000000
5      Brunei Darussalam  1980.0     BCG     99.000000
6               Bulgaria  1980.0     BCG     99.000000
7               Bulgaria  1980.0    POL3     99.000000
8                   Cuba  1980.0     BCG     99.000000
9                   Cuba  1980.0    POL3     99.000000
10               Denmark  1980.0    POL3     99.000000
11               Hungary  1980.0     BCG     99.000000
12               Hungary  1980.0  DTPCV1     99.000000
13               Hungary  1980.0  DTPCV3     99.000000
14               Hungary  1980.0    MCV1     99.000000
15               Iceland  1980.0  DTPCV1     99.000000
16               Iceland  1980.0  DTPCV3     99.000000
17        

In [11]:
conn = sqlite3.connect(DB)
q = """
SELECT c.country_name, i.year, i.disease, SUM(i.cases) AS total_cases
FROM incidence i
JOIN countries c ON i.iso3 = c.iso3
WHERE i.cases IS NOT NULL
GROUP BY c.country_name, i.year, i.disease
ORDER BY total_cases DESC
LIMIT 10;
"""
print(pd.read_sql(q, conn))
conn.close()

     country_name  year  disease  total_cases
0          Global  2023  TYPHOID    4583555.0
1          Global  1981  MEASLES    4078455.0
2          Global  1980  MEASLES    3852242.0
3  African Region  2023  TYPHOID    3635591.0
4          Global  1982  MEASLES    3623758.0
5          Global  1983  MEASLES    3586101.0
6          Global  1984  MEASLES    3026973.0
7          Global  2022  TYPHOID    2897152.0
8          Global  1985  MEASLES    2819553.0
9  African Region  2022  TYPHOID    2478436.0


In [12]:
conn = sqlite3.connect(DB)
q = """
SELECT v.year, v.antigen, 
       AVG(v.coverage_pct) AS avg_coverage, 
       AVG(i.incidence_rate) AS avg_incidence
FROM vaccination v
JOIN incidence i ON v.iso3 = i.iso3 AND v.year = i.year
GROUP BY v.year, v.antigen
ORDER BY v.year;
"""
print(pd.read_sql(q, conn))
conn.close()

      year       antigen  avg_coverage  avg_incidence
0     1980           BCG     54.111465     837.201511
1     1980        DTPCV1     63.396368     750.091502
2     1980        DTPCV3     45.304020     812.017859
3     1980         HEPB3           NaN     801.615945
4     1980          HIB3           NaN     801.615945
...    ...           ...           ...            ...
1103  2023         TTCV5     79.129577      80.068095
1104  2023         TTCV6     77.754335      80.068095
1105  2023  TYPHOID_CONJ     72.711094      80.068095
1106  2023          VAD1    122.384377      80.068095
1107  2023           YFV     71.091431      94.114923

[1108 rows x 4 columns]


In [13]:
conn = sqlite3.connect(DB)
q = """
SELECT c.country_name, v.year, v.coverage_pct
FROM vaccination v
JOIN countries c ON v.iso3 = c.iso3
WHERE v.antigen = 'MCV1' AND v.coverage_pct >= 95
ORDER BY v.year DESC, c.country_name;
"""
print(pd.read_sql(q, conn))
conn.close()

      country_name  year  coverage_pct
0          Algeria  2023         98.73
1          Algeria  2023         99.00
2          Algeria  2023         98.73
3          Algeria  2023         99.00
4          Andorra  2023         99.00
...            ...   ...           ...
13701      Hungary  1980         99.00
13702      Hungary  1980         99.00
13703      Hungary  1980         99.00
13704      Hungary  1980         99.00
13705      Hungary  1980         99.00

[13706 rows x 3 columns]
